In [ ]:
/*
 * Function Objects (functors) --> class with operator()
 *
 * Example:
 */
class X {
   public:
   void operator()(string str) { 
      cout << "Calling functor X with parameter " << str<< endl;
   } 

    operator string() const{return "X";} // type conversion function
};

int main()
{
   X foo;
   foo("Hi");    // Calling functor X with parameter Hi
}
/*
 * Benefits of functor:
 * 1. Smart function: capabilities beyond operator()
 * 	It can remember state. --> it is a class ,can have class member
 * 2. It can have its own type. --> ??
 */

In [ ]:
/*
 * Parameterized Function
 */
class X {
   public:
   X(int i) {}
   void operator()(string str) { 
      cout << "Calling functor X with parameter " << str<< endl;
   }
};

int main()
{
   X(8)("Hi");
}

In [ ]:
/*
 * Different schemes implement for Parameterized Function
 */

// how to make `add2` parameterized
void add2(int i) {
   cout << i+2 << endl;
}

// UC1: config outside of function
int val =2;

void add2(int i) {
   cout << i+val << endl;
}

int main()
{
   vector<int> vec = { 2, 3, 4, 5};   
   for_each(vec.begin(), vec.end(), add2); // {4, 5, 6, 7}
}

// UC2: nontype template parameter  
template<int val>
void addVal(int i) {
   cout << val+i << endl;
}
int main()
{  
   // the template parameter must be instanced
   for_each(vec.begin(), vec.end(), addVal<2>); // {4, 5, 6, 7}
   
   // --> can not complie, template generate code in compile time
   // int x = 2;
   // for_each(vec.begin(), vec.end(), addVal<x>); 

}

// UC3: functor
class AddValue {
   int val;
   public:
   AddValue(int j) : val(j) { }
   void operator()(int i) {
      cout << i+val << endl;
   }
};

int main()
{
   vector<int> vec = { 2, 3, 4, 5};   
   for_each(vec.begin(), vec.end(), AddValue(x)); // {4, 5, 6, 7}
}

In [ ]:
/*
 * Build-in Functors
 *
 * less greater  greater_equal  less_equal  not_equal_to
 * logical_and  logical_not  logical_or
 * multiplies minus  plus  divide  modulus  negate
 */

int x = multiplies<int>()(3,4);  //  x = 3 * 4 

if (not_equal_to<int>()(x, 10))   // if (x != 10)
   cout << x << endl;

In [ ]:
/*
 * Parameter Binding --> std::binding
 */
set<int> myset = { 2, 3, 4, 5};   
vector<int> vec;

// Multiply myset's elements by 10 and save in vec:
transform(myset.begin(), myset.end(),    // source
	      back_inserter(vec),              // destination
			bind(multiplies<int>(), placeholders::_1, 10));  // functor
    // First parameter of multiplies<int>() is substituted with myset's element
    // vec: {20, 30, 40, 50}


void addVal(int i, int val) {
   cout << i+val << endl;
}
for_each(vec.begin(), vec.end(), bind(addVal, placeholders::_1, 2));

In [ ]:
/*
 * Convert a regular function to a functor --> std::function
 */
double Pow(double x, double y) {
	return pow(x, y);
}

int main()
{
  set<int> myset = {3, 1, 25, 7, 12};
  deque<int> d;
  auto f = function<double (double,double)>(Pow);   //C++ 11
  transform(myset.begin(), myset.end(),     // source
		      back_inserter(d),              // destination
				bind(f, placeholders::_1, 2));  // functor
            //  d: {1, 9, 49, 144, 625}
}

In [ ]:

/*
 * Lambda
 */
set<int> myset = {3, 1, 25, 7, 12};
// when (x > 20) || (x < 5),  copy from myset to d
deque<int> d;

bool needCopy(int x){
   return (x>20)||(x<5);
}


transform(myset.begin(), myset.end(),     // source
          back_inserter(d),               // destination
          needCopy
          );

// C++ 11 lambda function:
transform(myset.begin(), myset.end(),     // source
          back_inserter(d),              // destination
          [](int x){return (x>20)||(x<5);}
          );

In [ ]:
/*
 * Why do we need functor in STL?
 * 
 * for example, set(sort by value), but what is the criteria do they use to sort? also like priority-queue, we need define which type of priority
 * 
 */

// the defination of set template class 
//* template<class _Kty,class _Pr = less<_Kty>,class _Alloc = allocator<_Kty> >
//* class set{}

set<int> myset = {3, 1, 25, 7, 12}; // myset: {1, 3, 7, 12, 25}
// default class _Pr = less<_Kty>, same as:
set<int, less<int> > myset = {3, 1, 25, 7, 12};

// set 传入参数不能为函数名
bool lsb_less(int x, int y) {
      return (x%10)<(y%10);
}

// set 传入参数为class类型 --> class _Pr = less<_Kty>
class Lsb_less {
   public:
   bool operator()(int x, int y) {
      return (x%10)<(y%10);
   }
};
int main()
{
  set<int, Lsb_less> myset = {3, 1, 25, 7, 12};  // myset: {1,12,3,25,7}
}

In [ ]:
/*
 * Predicate
 *
 * A functor or function that:
 * 1. Returns a boolean
 * 2. Does not modify data
 */

class NeedCopy {
   bool operator()(int x){   
      return (x>20)||(x<5);  
   }
};

transform(myset.begin(), myset.end(),     // source
          back_inserter(d),               // destination
          NeedCopy()
          );

// Predicate is used for comparison or condition check